In [1]:
import sys
from pathlib import Path

parent = str(Path().resolve().parent)
if parent not in sys.path:
    sys.path.insert(0, parent)

In [2]:
import re
import numpy as np
import ast
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [3]:
df_groups = pd.read_csv("../data/wip/groups_wMembers3.csv")
#df_groups["all_members"] = df_groups["all_members"].apply(ast.literal_eval)

In [4]:
df_groups = df_groups[["title", "all_members"]]
df_groups

,title,all_members
0,A Friend in London,"['Aske Damm Bramming', 'Esben Svane', 'Sebasti..."
1,Ache,"['Finn Olafsson', 'Per Wium', 'Peter Mellin', ..."
2,ADS (band),"['Karsten Hjarsø', 'Lars Top-Galia', 'Martin K..."
3,Agathorn,[]
4,Alabama Black Snakes,"['Gustav Pontoppidan', 'Hank Lee Tuffer', 'Jon..."
...,...,...
1029,Odense Symfoniorkester,"['Alf Vestergaard', 'Anca Bold', 'Angelika Wag..."
1030,Sjællands Symfoniorkester,"['Fritz O. Jacobsen', 'Harry Gram-Jensen', 'Il..."
1031,Sønderjyllands Symfoniorkester,"['Anne Augustinsson', 'Brian Quist Jørgensen',..."
1032,Aalborg Symfoniorkester,"['Bettina Ejlerts Jensen', 'Bo Juel Christians..."


### Group Musician Graph

In [5]:
df_groups["edgelist"] = df_groups.apply(lambda row: [(row["title"], member) for member in ast.literal_eval(row["all_members"])], axis=1)
edgelist = [edge for sublist in df_groups["edgelist"] for edge in sublist]

In [6]:
df_groups

,title,all_members,edgelist
0,A Friend in London,"['Aske Damm Bramming', 'Esben Svane', 'Sebasti...","[(A Friend in London, Aske Damm Bramming), (A ..."
1,Ache,"['Finn Olafsson', 'Per Wium', 'Peter Mellin', ...","[(Ache, Finn Olafsson), (Ache, Per Wium), (Ach..."
2,ADS (band),"['Karsten Hjarsø', 'Lars Top-Galia', 'Martin K...","[(ADS (band), Karsten Hjarsø), (ADS (band), La..."
3,Agathorn,[],[]
4,Alabama Black Snakes,"['Gustav Pontoppidan', 'Hank Lee Tuffer', 'Jon...","[(Alabama Black Snakes, Gustav Pontoppidan), (..."
...,...,...,...
1029,Odense Symfoniorkester,"['Alf Vestergaard', 'Anca Bold', 'Angelika Wag...","[(Odense Symfoniorkester, Alf Vestergaard), (O..."
1030,Sjællands Symfoniorkester,"['Fritz O. Jacobsen', 'Harry Gram-Jensen', 'Il...","[(Sjællands Symfoniorkester, Fritz O. Jacobsen..."
1031,Sønderjyllands Symfoniorkester,"['Anne Augustinsson', 'Brian Quist Jørgensen',...","[(Sønderjyllands Symfoniorkester, Anne Augusti..."
1032,Aalborg Symfoniorkester,"['Bettina Ejlerts Jensen', 'Bo Juel Christians...","[(Aalborg Symfoniorkester, Bettina Ejlerts Jen..."


In [7]:
GroupMusicianG = nx.Graph()
GroupMusicianG.add_edges_from(edgelist)

In [8]:
len(GroupMusicianG.nodes), len(GroupMusicianG.edges)

(4931, 4645)

In [9]:
list_of_connected_comp = list(nx.connected_components(GroupMusicianG))
print(len(list_of_connected_comp)) # number of connected components

480


In [10]:
sorted_list = sorted(list_of_connected_comp, key=len, reverse = True)
lengths = [len(c) for c in sorted_list]

### Group graph

In [11]:
all_groups = list([group for group in df_groups["title"].values if group in GroupMusicianG.nodes])
GroupsG = nx.projected_graph(GroupMusicianG, all_groups, multigraph=False)

In [12]:
# Save graph
#with open("../data/wip/groups_graph.pkl", "wb") as f:
#    pickle.dump(GroupsG, f)

### Musician Graph

In [13]:
df_groups["all_members"].values

array(["['Aske Damm Bramming', 'Esben Svane', 'Sebastian Vinther Olsen', 'Tim Schou']",
       "['Finn Olafsson', 'Per Wium', 'Peter Mellin', 'Steen Toft Andersen', 'Torsten Olafsson']",
       "['Karsten Hjarsø', 'Lars Top-Galia', 'Martin Krogh', 'Michael Funder Thorlasius', 'Peter Top-Jensen', 'Stig Pedersen', 'Søren Slys Johnsen']",
       ...,
       "['Anne Augustinsson', 'Brian Quist Jørgensen', 'Henrik Steensgaard', 'Jesper Hindø', 'Karsten Dalsgaard Madsen', 'Kristjan Orii Sigerleifsson', 'Lars Rimer', 'Niels Christian Lund Mathiesen', 'Nikolai Skliarevski', 'Peder Linneberg', 'Rumen Lukanov']",
       "['Bettina Ejlerts Jensen', 'Bo Juel Christiansen', 'Claus Ettrup', 'Erik Sandberg (3)', 'Hanne Høy Houengaard', 'Ib Dirkov', 'Ida Marie Sørmo', 'Jacob Ringsmose', 'Jenny Sjöberg', 'Jette Rosendal', 'Judith Blauw', 'Karen Kjelds', 'Leah Aksnes', 'Lisbeth Binderup-Thordal', 'Marius Ungureanu', 'Mette Nielsen', 'Michael Hübner', 'Michael Pilgaard', 'Moshe Atzmon', 'Ruben Kristensen

In [14]:
all_musicians = list(np.unique([
    str(member)
    for row_list in df_groups["all_members"].values
    for member in ast.literal_eval(row_list)
    if member in GroupMusicianG.nodes and member != ''
]))
all_musicians = [str(member) for member in all_musicians]

MusiciansG = nx.projected_graph(GroupMusicianG, all_musicians, multigraph=False)

In [15]:
# Save graph
#with open("../data/wip/Musicians_graph.pkl", "wb") as f:
#    pickle.dump(MusiciansG, f)